- Update: 2023.03.13

# 0. 配置参数

- `dict_page`：品类链接的参数
- `out`：保存结果的路径

In [1]:
dict_page = {'url': 'https://www.walmart.com/search?q=fram+cabin+air+filter&sort=price_low&page=WHAT_THE_FUCK&affinityOverride=default',
             'max_page': 3}
# pg.position()
position_url = (200, 60)
position_html = (280, 195)

out = '../../file/walmart-fram_cabin_air_filter-menuWHAT_THE_FUCK.xlsx'

In [2]:
import pyautogui as pg
import time
import pyperclip as pc
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
pg.PAUSE = 0.3
pg.FAILSAFE = True

In [3]:
def get_html(url):
    pg.moveTo(position_url)
    pg.click()
    pg.hotkey('ctrl', 'a')
    pg.press('delete')
    
    pc.copy(url)
    pg.hotkey('ctrl', 'v')
    pg.press('enter')
    time.sleep(10)
    
    pg.press('f12')
    time.sleep(1.5)
    
    pg.moveTo(position_html)
    pg.click()
    pg.hotkey('ctrl', 'c')
    
    pg.press('f12')
    
    return pc.paste()
    
def parse_html(html):
    page = BeautifulSoup(html)
    
    list_item = []
    list_url = []
    list_article = page.find_all('div', attrs={'class': 'mb1 ph1 pa0-xl bb b--near-white w-25'})
    list_article += page.find_all('div', attrs={'class': 'mb1 ph1 pa0-xl bb b--near-white w-25 pb1-xl'})
    for article in list_article:
        list_item.append(article.find('div', attrs={'class', 'sans-serif mid-gray relative flex flex-column w-100 hide-child-opacity'})['data-item-id'])
        list_url.append('https://www.walmart.com' + article.find('a', attrs={'class', 'absolute w-100 h-100 z-1 hide-sibling-opacity'})['href'])
    
    pd.DataFrame({'item': list_item, 'url': list_url}).to_excel(out.replace('WHAT_THE_FUCK', str(i+1)), index=False)

In [4]:
for i in range(dict_page['max_page']):
    html = get_html(dict_page['url'].replace('WHAT_THE_FUCK', str(i+1)))
    time.sleep(2)
    parse_html(html)

In [5]:
list_df = []
for i in range(dict_page['max_page']):
    list_df.append(pd.read_excel(out.replace('WHAT_THE_FUCK', str(i+1)), header=0))
    
df = pd.concat(list_df).reset_index(drop=True)

df['图片链接'] = ''
df['标题'] = ''
df['价格'] = ''
df['品牌'] = ''
df['型号'] = ''
df['工厂'] = ''
df['工厂号'] = ''
df['原价'] = ''

df

,item,url
0,14XP573XLNIX,https://www.walmart.com//ip/FRAM-Ultra-Air-XGA...
1,0V5DWSTYW8DO,https://www.walmart.com//ip/FRAM-Ultra-Air-XGA...
2,214UXF52FO7E,https://www.walmart.com//ip/FRAM-Ultra-Air-XGA...
3,1NKZQSK73CZQ,https://www.walmart.com//ip/FRAM-Ultra-Air-XGA...
4,4JLVIB2OWF83,https://www.walmart.com//ip/FRAM-Ultra-Air-XGA...
...,...,...
995,20ETMOAY1MW5,https://www.walmart.com//ip/VIBRANT-10931-Air-...
996,26Y4I95V2WB1,https://www.walmart.com//ip/No-Toil-Super-Flo-...
997,3B3B2T1N0R6Q,https://www.walmart.com//ip/No-Toil-Super-Flo-...
998,0TMK1PBM95W5,https://www.walmart.com//ip/VIBRANT-10925-Air-...


In [6]:
df.to_excel(out.replace('menuWHAT_THE_FUCK', 'content0'), index=False)